In [308]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc
import matplotlib.pyplot as plt
import sklearn
import scipy.sparse 
from sklearn.impute import KNNImputer
from scipy.stats import t
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import  lightgbm as lgbm
import warnings
warnings.filterwarnings('ignore')


In [309]:
maindir = "../input/zindi-blood-ai" # Directory with your files
traincsv = maindir+"/Update_train.csv"
testcsv = maindir+"/Updated_Test.csv"
train = pd.read_csv('../input/zindi-blood-ai/Update_train.csv')
test = pd.read_csv('../input/zindi-blood-ai/Updated_Test.csv')
targets = ['hdl_cholesterol_human','hemoglobin(hgb)_human','cholesterol_ldl_human']
# train.loc[train[targets[0]]=='low',targets[0]] = 'not_ok'
# train.loc[train[targets[0]]=='ok',targets[0]] = 0
# train.loc[train[targets[0]]=='high',targets[0]] = 'not_ok'
# train.loc[train[targets[1]]=='low',targets[1]] = -1
# train.loc[train[targets[1]]=='ok',targets[1]] = 0
# train.loc[train[targets[1]]=='high',targets[1]] = 1
# train.loc[train[targets[2]]=='low',targets[2]] = -1
# train.loc[train[targets[2]]=='ok',targets[2]] = 0
# train.loc[train[targets[2]]=='high',targets[2]] = 1
train['target'] = train[targets[0]]
#.agg('_'.join, axis=1)

In [310]:
def feature_ing(df):
#     means = df.groupby('donation_id')[fea].agg(np.mean).reset_index()
#     stds = df.groupby('donation_id')[fea].agg(np.std).reset_index()
#     maxs = df.groupby('donation_id')[fea].agg(np.max).reset_index()
#     mins = df.groupby('donation_id')[fea].agg(np.min).reset_index()
    absorb = ['absorbance'+str(i) for i in range(0,170)]
    mins1 = df.groupby('donation_id')[absorb[:17]].agg(min).min(axis = 1).reset_index()
    idxmins = df.groupby('donation_id')[absorb[:17]].agg(min).idxmin(axis = 1).reset_index()
#
    maxs1 = df.groupby('donation_id')[absorb[:17]].agg(max).max(axis = 1).reset_index()
    idxmaxs = df.groupby('donation_id')[absorb[:17]].agg(max).idxmax(axis = 1).reset_index()
#     temps = 
#     print(df.columns)
#          temps = 
#     print(df.columns)
    for i in range(2,11):
        
        mins1 = mins1.merge(df.groupby('donation_id')[absorb[17*(i-1):17*i]].agg(min).min(axis = 1).reset_index(),on = 'donation_id', how = 'left' , suffixes = (None,'_window'+str(i)))    
        idxmins = idxmins.merge(df.groupby('donation_id')[absorb[17*(i-1):17*i]].agg(min).idxmin(axis = 1).reset_index(),on = 'donation_id', how = 'left' , suffixes = (None,'_window'+str(i)))    
#        res = res.merge(df[['donation_id','target','std']].copy().drop_duplicates(ignore_index = True),on = 'donation_id', how = 'left') 
        maxs1 = maxs1.merge(df.groupby('donation_id')[absorb[17*(i-1):17*i]].agg(max).max(axis = 1).reset_index(),on = 'donation_id', how = 'left' , suffixes = (None,'_window'+str(i)))    
        idxmaxs = idxmaxs.merge(df.groupby('donation_id')[absorb[17*(i-1):17*i]].agg(max).idxmax(axis = 1).reset_index(),on = 'donation_id', how = 'left' , suffixes = (None,'_window'+str(i)))    
    res = maxs1.merge(idxmaxs,on = 'donation_id', how = 'left', suffixes = ('_max','_idx_max'))
    res1 = mins1.merge(idxmins,on = 'donation_id', how = 'left', suffixes = ('_min','_idx_min'))
    res = res.merge(res1,on = 'donation_id',how = 'left' )
    res = res.merge(df[['donation_id','target','std']].copy().drop_duplicates(ignore_index = True),on = 'donation_id', how = 'left') 
    
    temp_maxidx =  df.groupby('donation_id')[['temperature','humidity']].apply(lambda x : x.idxmax(axis = 0)%60).reset_index()
    temp_max = df.groupby('donation_id')[['temperature','humidity']].agg(max).reset_index()
    temp_minidx =  df.groupby('donation_id')[['temperature','humidity']].apply(lambda x : x.idxmin(axis = 0)%60).reset_index()
    temp_min = df.groupby('donation_id')[['temperature','humidity']].agg(min).reset_index()
    temp_max = temp_max.merge(temp_maxidx,on = 'donation_id',how = 'left')
    temp_min = temp_min.merge(temp_minidx,on = 'donation_id',how = 'left')
    temp =   temp_max.merge(temp_min,on = 'donation_id',how = 'left')
    res = res.merge(temp, on = 'donation_id', how = 'left')
    #   res = df[['donation_id','target']].copy().drop_duplicates(ignore_index = True)
#     res = res.merge(means,on = 'donation_id',how = 'left',suffixes = (None,'_mean')).merge(stds,on = 'donation_id',how = 'left',suffixes = (None,'_std')).merge(maxs,on = 'donation_id',how = 'left',suffixes = (None,'_max')).merge(mins,on = 'donation_id',how = 'left',suffixes = (None,'_min'))
    return res

In [311]:


# df
# feature_ing(train).info()

In [312]:
# fea = ['absorbance'+str(i) for i in range(0,170)]+['std','temperature','humidity']
# train = feature_ing(train,fea)

In [313]:
means_col = ['absorbance'+str(i) for i in range(0,10)]+['std','temperature','humidity']
maxs_col = ['absorbance'+str(i)+'_max' for i in range(0,10)]+['std_max','temperature_max','humidity_max']
mins_col = ['absorbance'+str(i)+'_min' for i in range(0,10)]+['std_min','temperature_min','humidity_min']
stds_col = ['absorbance'+str(i)+'_std' for i in range(0,10)]+['std_std','temperature_std','humidity_std']
all_cols = means_col +mins_col + maxs_col + stds_col 

In [314]:
# fea
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [315]:
targ_fea = []
# fea = train.columns
# ['absorbance'+str(i) for i in range(10)]+['std','temperature','humidity']
# def feature_ing(df,fea,targ = True):
    
#     means = df.groupby('donation_id')[fea].agg(np.mean).reset_index()
#     stds = df.groupby('donation_id')[fea].agg(np.std).reset_index()
#     maxs = df.groupby('donation_id')[fea].agg(np.max).reset_index()
#     mins = df.groupby('donation_id')[fea].agg(np.min).reset_index()
#     cumsums = df.groupby('donation_id').agg(np.mean)[fea[:-3]].cumsum(axis = 1)
#     cumprods = df.groupby('donation_id').agg(np.mean)[fea[:-3]].cumprod(axis = 1)
#     if targ:
#         res = df[['donation_id','target']+targ_fea].copy().drop_duplicates(ignore_index = True)
#     else:
#         res = df[['donation_id','target']].copy().drop_duplicates(ignore_index = True)
#     res = res.merge(means,on = 'donation_id',how = 'left',suffixes = (None,'_mean')).merge(stds,on = 'donation_id',how = 'left',suffixes = (None,'_std')).merge(maxs,on = 'donation_id',how = 'left',suffixes = (None,'_max')).merge(mins,on = 'donation_id',how = 'left',suffixes = (None,'_min')).merge(cumprods,on = 'donation_id',how = 'left',suffixes = (None,'_cumprod')).merge(cumsums,on = 'donation_id',how = 'left',suffixes = (None,'_cumsum'))
#     return res
# df = train.copy()
test['target'] = 'ok'
train = feature_ing(train)
# df = test.copy()
test = feature_ing(test)
means_col = ['absorbance'+str(i) for i in range(0,170)]+['std','temperature','humidity']
maxs_col = ['absorbance'+str(i)+'_max' for i in range(0,170)]+['std_max','temperature_max','humidity_max']
mins_col = ['absorbance'+str(i)+'_min' for i in range(0,170)]+['std_min','temperature_min','humidity_min']
stds_col = ['absorbance'+str(i)+'_std' for i in range(0,170)]+['std_std','temperature_std','humidity_std']
cumprods_col = ['absorbance'+str(i)+'_cumprod' for i in range(0,170)]
cumsums_col = ['absorbance'+str(i)+'_cumsum' for i in range(0,170)]

all_cols = means_col +mins_col + maxs_col + stds_col + cumprods_col + cumsums_col# train[fea]
# t_h = ['temperature','humidity']
# train[t_h]
# test['target'] = 'ok'
# train = feature_ing(train,fea)
# test = feature_ing(test,fea)
num_fea = train.drop(['donation_id','target'],axis = 1).select_dtypes(include = 'float').columns.tolist()
cat_fea = train.drop(['donation_id','target'],axis = 1).select_dtypes(include = 'object').columns.tolist()

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
# pca = PCA(n_components = 0.999999)
# X = pca.fit_transform(train[fea])
# # RobustScaler is less prone to outliers.
# test_X = pca.transform(train[fea])
# y  = train['target']
# d_id = train['donation_id']
# d_id_tes = test['donation_id']
# train = pd.DataFrame(X,columns = ['feat_'+str(i) for i in range(88)])
# test = pd.DataFrame(test_X,columns = ['feat_'+str(i) for i in range(88)])
# train['target'] = y
# train['donation_id'] = d_id
# test['donation_id'] = d_id_tes
# fea = train.columns.tolist()[:-2]
rob_scaler = RobustScaler()
train[num_fea] = rob_scaler.fit_transform(train[num_fea].values)
test[num_fea] = rob_scaler.transform(test[num_fea])
from sklearn.preprocessing import LabelEncoder
for f in cat_fea:
    lab_enc = LabelEncoder()
    train[f] = lab_enc.fit_transform(train[f])
    test[f] = lab_enc.transform(test[f])

In [316]:
drop_fea = train.isna().sum()[train.isna().sum()>0].index.tolist()
train = train.drop(drop_fea, axis = 1)
test = test.drop(drop_fea, axis = 1)

In [317]:
# fea = 

In [318]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])
# train.drop(targets, axis =1 , inplace = True)

In [319]:
# train = reduce_mem_usage(train)
# train.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 0 to 485
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   donation_id         486 non-null    int64  
 1   0_max               486 non-null    float64
 2   0_window2_max       486 non-null    float64
 3   0_window3_max       486 non-null    float64
 4   0_window4_max       486 non-null    float64
 5   0_window5_max       486 non-null    float64
 6   0_window6_max       486 non-null    float64
 7   0_window7_max       486 non-null    float64
 8   0_window8_max       486 non-null    float64
 9   0_window9_max       486 non-null    float64
 10  0_window10_max      486 non-null    float64
 11  0_idx_max           486 non-null    int64  
 12  0_window2_idx_max   486 non-null    int64  
 13  0_window3_idx_max   486 non-null    int64  
 14  0_window4_idx_max   486 non-null    int64  
 15  0_window5_idx_max   486 non-null    int64  
 16  0_window

In [320]:
# train['id']
# train.info()

In [ ]:
n_folds = 5
Experiment_name = 'model1'
'''creating folds '''
os.makedirs("proc_data", exist_ok=True)
groups_by_patient_id_list = train.donation_id.tolist()
try: 
    folds=pd.read_csv("./proc_data/folds_id.csv")
    train=train.merge(folds,on="donation_id",how="left")
    train.fold.nunique()
except: 
    from sklearn.model_selection import GroupKFold 
    kfold=GroupKFold(n_splits = n_folds) # change this random_state or all of you will have the same score  :D 
    y_labels = train['target']
    train.reset_index(drop=True,inplace=True)
    folds=train[["donation_id"]].copy()
    folds["fold"]=0
    for fold,(tr_indx,val_ind) in enumerate(kfold.split(train, y_labels, groups = groups_by_patient_id_list)) : 
        folds.loc[val_ind,"fold"]=fold
    folds.to_csv("./proc_data/folds_id.csv",index=False)
    train=train.merge(folds,on="donation_id",how="left")

    del folds

target_name="target"
id_name="donation_id"
features_to_remove=[target_name,id_name,"fold","donation_id"]
features=train.columns.tolist()
features=[ f for f in  features if f not in features_to_remove  ]
'''scaling the data'''
# scaler = StandardScaler()
# unscaled_train = train
# train[features] = scaler.fit_transform(unscaled_train[features])
# test_unscaled=test
# test[features] = scaler.transform(test_unscaled[features])
'''modelisation part'''
from sklearn.metrics import accuracy_score
def metric(y,x):
    return accuracy_score(x,y)
def train_function(model,train,test,params,other_params,target_name,features,metric):
    folds_num=train.fold.nunique()
    validation=train[["donation_id","fold",target_name]].copy()
    validation["pred_"+target_name]=0
    sub=test[["donation_id"]].copy()
    sub[target_name]=0
    for fold in np.sort(train.fold.unique()):
        print("#"*50+" {} ".format(fold)+"#"*50)
        os.makedirs("model_save/lgbm/{}/{}".format(Experiment_name,str(int(fold))), exist_ok=True)
        X_train=train[train.fold!=fold]
        X_val=train[train.fold==fold]

        train_pred,validation_pred,test_pred,model_save=model(X_train,X_val,test,params,other_params)
#         print(.shape)
        validation.loc[validation.fold==fold,"pred_"+target_name]= np.argmax(validation_pred,axis = 1)
        sub[target_name]+=np.argmax(test_pred,axis = 1)/folds_num
        train_score=metric(X_train[target_name],np.argmax(train_pred,axis = 1))
        val_score=metric(X_val[target_name],np.argmax(validation_pred,axis = 1))
        print("train score : {} validation score : {}".format(round(train_score,4),round(val_score,4)))
    final_validation_score=metric(validation[target_name],validation["pred_"+target_name])
    print("final validation score : {}".format(final_validation_score))

    return sub,validation,final_validation_score,model_save

def lgbm_model(X_train,X_val,X_test,params,other_params):
    dtrain = lgbm.Dataset(data=X_train[features], label=X_train[target_name], feature_name=features)
    dval = lgbm.Dataset(data=X_val[features], label=X_val[target_name], feature_name=features)

    model = lgbm.train(
        params=params,
        train_set=dtrain,
        num_boost_round=other_params["num_boost_round"],
        valid_sets=(dtrain, dval),
        early_stopping_rounds=other_params["early_stopping_rounds"],
        verbose_eval=other_params["verbose_eval"],
    )        
    best_iteration = model.best_iteration
    train_pred=model.predict(X_train[features], num_iteration=best_iteration)
    validation_pred=model.predict(X_val[features], num_iteration=best_iteration)
    test_pred=model.predict(test[features], num_iteration=best_iteration)

    return train_pred,validation_pred,test_pred,model
other_params={"num_boost_round":10000,
          "early_stopping_rounds":1000,
          "verbose_eval":1000,
}
# n_classes = y_labels.unique().shape[0]
lgbm_params = {'task': 'train',
    'boosting_type': 'dart',
    'objective': 'multiclassova',
#     'num_class':3,
#     'is_unbalance': True,
    'metric': 'multi_error',
    'learning_rate': 0.02296,
    'max_depth': 17,
    'num_leaves': 13,
    'feature_fraction': 0.9,
#     'importance_type':'gain',
    'bagging_fraction': 0.6,
    'bagging_freq': 17,
    'class_weight' : 'balanced',
#     "bagging_fraction": 0.2,
#     "bagging_freq": 2,
    "num_classes" :3,
    "verbosity"  : -1,
#     "boosting_type": "gbdt",
#     "feature_fraction": 0.6,
#     "learning_rate": 0.01,
#     "max_depth": 5,
#     "num_threads": 2,
#     "objective": "multiclassova",
#     "metric": "accuracy",
    "seed": 2020
#     "lambda_l1" : 15,
#     "lambda_l2" : 4, # this was 5
}
sub,validation,score,model=train_function(model=lgbm_model,
                                    train=train,
                                    test=test,
                                    params=lgbm_params,
                                    other_params=other_params,
                                    target_name=target_name,
                                    features=features,
                                    metric=metric)
# return sub, validation, score, model

################################################## 0 ##################################################
[1000]	training's multi_error: 0.0335052	valid_1's multi_error: 0.489796
[2000]	training's multi_error: 0	valid_1's multi_error: 0.469388
[3000]	training's multi_error: 0	valid_1's multi_error: 0.44898
[4000]	training's multi_error: 0	valid_1's multi_error: 0.469388
[5000]	training's multi_error: 0	valid_1's multi_error: 0.469388
[6000]	training's multi_error: 0	valid_1's multi_error: 0.489796
[7000]	training's multi_error: 0	valid_1's multi_error: 0.459184
[8000]	training's multi_error: 0	valid_1's multi_error: 0.44898
[9000]	training's multi_error: 0	valid_1's multi_error: 0.44898
[10000]	training's multi_error: 0	valid_1's multi_error: 0.459184
train score : 1.0 validation score : 0.5408
################################################## 1 ##################################################
[LightGBM] [Warning] Unknown parameter: class_weight
[1000]	training's multi_error: 0.020565

In [ ]:
# train['target']


In [ ]:
# # validation
# feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,train[fea].columns)), columns=['Value','Feature'])

# plt.figure(figsize=(20, 10))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.show()
cust_fea = pd.DataFrame({'features': train[features].columns ,
                            'importances' : model.feature_importance() })
cust_fea = cust_fea.loc[cust_fea.importances>0].features.tolist()

In [ ]:
# len(cust_fea)
(validation['pred_target']).unique()

In [ ]:
train.loc[train.target ==0]